In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
sns.set(style= 'darkgrid')

In [3]:
data = pd.read_csv('AdvertAndSales.csv')
data.head()

,TV,Radio,Social Media,Influencer,Sales
0,16.0,6.566231,2.907983,Mega,54.732757
1,13.0,9.237765,2.409567,Mega,46.677897
2,41.0,15.886446,2.913410,Mega,150.177829
3,83.0,30.020028,6.922304,Mega,298.246340
4,15.0,8.437408,1.405998,Micro,56.594181


In [4]:
data.isnull().sum().sort_values(ascending=False)

TV              10
Social Media     6
Sales            6
Radio            4
Influencer       0
dtype: int64

In [5]:
for i in data.columns:
    if ((data[i].isnull().sum() / len(data)) * 100).round(2) < 30:
        if data[i].dtypes == 'O':
            data[i].fillna(data[i].mode()[0], inplace = True)
        else:
            data[i].fillna(data[i].median(), inplace = True)
    else:
        data.drop(i, axis = 1, inplace = True)

data.isnull().sum().sort_values(ascending=False)

TV              0
Radio           0
Social Media    0
Influencer      0
Sales           0
dtype: int64

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4572 entries, 0 to 4571
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TV            4572 non-null   float64
 1   Radio         4572 non-null   float64
 2   Social Media  4572 non-null   float64
 3   Influencer    4572 non-null   object 
 4   Sales         4572 non-null   float64
dtypes: float64(4), object(1)
memory usage: 178.7+ KB


<h3><b>Data Preprocessing<hr></b></h3>

In [7]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
import joblib

df = data.copy()

for i in df.drop("Sales", axis=1).columns:
    if i in df.select_dtypes(include= 'number').columns:
        scaler = StandardScaler()
        df[i] = scaler.fit_transform(df[[i]])
        joblib.dump(scaler, open(f'{i}.scaler.pkl', 'wb'))
    else:
        encoder = LabelEncoder()
        df[i] = encoder.fit_transform(df[[i]])
        joblib.dump(encoder, open(f'{i}.encoder.pkl', 'wb'))
df.head()

,TV,Radio,Social Media,Influencer,Sales
0,-1.458765,-1.198745,-0.187979,1,54.732757
1,-1.573736,-0.922522,-0.413405,1,46.677897
2,-0.500678,-0.235083,-0.185524,1,150.177829
3,1.108909,1.226255,1.627637,1,298.246340
4,-1.497089,-1.005275,-0.867304,2,56.594181


<h3><b>Modelling<hr></b></h3>

In [14]:
x = df.drop('Sales', axis=1)
y = df.Sales


from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.25, random_state = 40)

trains = pd.concat([xtrain, ytrain], axis=1)
tests = pd.concat([xtest, ytest], axis=1)

display(xtest.head())
display(ytest.head ())

,TV,Radio,Social Media,Influencer
1740,-0.194090,-0.428411,-1.139247,1
4416,-1.113854,-0.865625,-1.275407,0
1226,-0.270737,-0.063170,0.121957,3
2848,-0.653972,-1.300665,0.462995,0
1826,-1.458765,-1.831002,0.097776,1


1740    178.601661
4416     92.848374
1226    166.124099
2848    134.445973
1826     56.016896
Name: Sales, dtype: float64

In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

model = RandomForestRegressor()

model.fit(xtrain, ytrain)
val = model.predict(xtrain)

print(f"Model Cross Validation is: {round(r2_score(val, ytrain)*100, 2)}\n")

pred = model.predict(xtest)
print(f"Model Performance is: {round(r2_score(ytest, pred)*100, 2)}")

Model Cross Validation is: 99.92

Model Performance is: 99.24


In [20]:
# save the model

joblib.dump(model, open(f'advertModel.pkl', 'wb'))

